# XML Writer – Demo

Dieses Notebook zeigt die Verwendung von `graphfw.io.writers.xml_writer` mit dem Namensschema:

`<prefix>[_<YYYYMMDD>_<hhmmss>][_<postfix>].xml`

**Ziele:**
- DataFrame erstellen
- XML-Dateipfad vorschauen (`build_xml_path`)
- XML exportieren (`write_xml`)
- Ergebnis prüfen und optional wieder einlesen (`pandas.read_xml`)


In [7]:
# Imports & Setup
from pathlib import Path
import pandas as pd
import sys

import sys
sys.path.insert(0, "..")  # eine Ebene hoch zum Repo-Root

from graphfw.io.writers.xml_writer import build_xml_path, write_xml

pd.__version__

'2.3.2'

## Beispiel-Daten vorbereiten

In [8]:
df = pd.DataFrame(
    {
        "Id": [1001, 1002, 1003],
        "Title": ["Alpha", "Beta", "Gamma"],
        "Amount": [199.90, 49.00, 325.50],
        "Created": pd.to_datetime(["2025-09-12", "2025-09-12", "2025-09-12"]) ,
    }
)
df

,Id,Title,Amount,Created
0,1001,Alpha,199.9,2025-09-12
1,1002,Beta,49.0,2025-09-12
2,1003,Gamma,325.5,2025-09-12


## Pfadvorschau erzeugen (`build_xml_path`)
Der Pfad wird **nicht** angelegt, sondern nur gemäß Schema generiert.

In [3]:
preview_with_ts = build_xml_path(prefix="Export", postfix="Orders", timestamp=True)
preview_no_ts  = build_xml_path(prefix="Export", postfix="Orders", timestamp=False)
preview_with_ts, preview_no_ts

(WindowsPath('c:/Users/erhard.rainer/Documents/GitHub/GRAPH_API/notebooks/Export_20250912_020116_Orders.xml'),
 WindowsPath('c:/Users/erhard.rainer/Documents/GitHub/GRAPH_API/notebooks/Export_Orders.xml'))

## XML schreiben (`write_xml`)
Standardmäßig wird im **aktuellen Arbeitsverzeichnis (cwd)** gespeichert.

Parameter-Highlights:
- `timestamp=True` fügt `YYYYMMDD_hhmmss` **zwischen** `prefix` und `postfix` ein
- `overwrite=False` erzeugt bei Kollision automatisch `_001`, `_002`, ...

In [10]:
xml_path_1 = write_xml(
    df,
    prefix="Export",
    postfix="Orders",
    timestamp=True,
    directory=r"C:\Users\erhard.rainer\Documents\GitHub\GRAPH_API\_output",
    encoding="utf-8",
    index=False,
    root_name="items",
    row_name="item",
    xml_declaration=True,
    pretty_print=True,
    overwrite=False,
)
xml_path_1

TypeError: write_xml() got an unexpected keyword argument 'directory'

## Datei inspizieren (erste Zeilen anzeigen)

In [ ]:
text = Path(xml_path_1).read_text(encoding="utf-8")
print("\n".join(text.splitlines()[:20]))

## Ohne Zeitstempel (stabiler Name)
Bei `timestamp=False` lautet der Name `prefix[_postfix].xml`. Existiert die Datei bereits und `overwrite=False`, wird automatisch `_001`, `_002`, … angehängt.

In [ ]:
xml_path_2 = write_xml(
    df,
    prefix="Export",
    postfix="Orders",
    timestamp=False,
    overwrite=False,
)
xml_path_2

## Optional: XML wieder als DataFrame einlesen
`pandas.read_xml` ist ab pandas 1.3 verfügbar. Je nach Struktur der XML kann `xpath` gesetzt werden.


In [ ]:
try:
    df_back = pd.read_xml(xml_path_1)
    display(df_back.head())
except Exception as e:
    print("Hinweis: Konnte XML nicht direkt lesen:", e)
    print("Tipp: Mit 'xpath' und passenden Root-/Row-Namen erneut versuchen, z. B. xpath='.//item'")

## Zusammenfassung
- **Schema:** `<prefix>[_<YYYYMMDD>_<hhmmss>][_<postfix>].xml`
- **Rückgabe:** vollständiger Pfad der erzeugten Datei
- **Robustheit:** automatischer eindeutiger Suffix bei `overwrite=False`
- **Tipps:** `encoding='utf-8'`, sprechende `root_name`/`row_name` setzen
